### I. Research Questions:
1. Which psychiatric diagnosis are associated with higher ICU readmission rates? -> Do patient with mood disorders (296.x), psychotic disorders (295.x) or substance use (303-305) have higher [30]-day readmission rates compared to those without?
2. Does discontinuation or poor psychotropic medications at discharge predict higher readmissions? -> Does the discontinuation of psychotropic medication upon discharge correlate with increased readmission risk?
3. Can we combine patient-level features to predict ICU readmission risk? -> which combination of demographics, ICU, severity scores and medication changes is most predictive of readmission?

In [1]:
import pandas as pd

admissions = pd.read_csv('./data/hosp/admissions.csv')
patients = pd.read_csv('./data/hosp/patients.csv')
diagnoses = pd.read_csv('./data/hosp/diagnoses_icd.csv')
icustays = pd.read_csv('./data/icu/icustays.csv')
prescriptions = pd.read_csv('./data/hosp/prescriptions.csv')

print("Initial admissions:", admissions.shape[0], "Patients:", patients['subject_id'].nunique(), "Notes:", diagnoses.shape[0])

# Exclude patients under 18
patients_adult = patients[patients['anchor_age'] >= 18]
admissions = admissions[admissions['subject_id'].isin(patients_adult['subject_id'])].copy()
diagnoses = diagnoses[diagnoses['hadm_id'].isin(admissions['hadm_id'])].copy()

# Recalculate patients and notes counts
print("After excluding <18:", admissions.shape[0],
      "Patients:", admissions['subject_id'].nunique(),
      "Notes:", diagnoses.shape[0])

# Identify psychiatric conditions
# Updated list of ICD codes for mental disorders
psych_icd_codes = ('296', '295', '303', '304', '305', 'F20', 'F21', 'F22', 'F23', 'F24', 'F25',
                           'F28', 'F29', 'F30', 'F31', 'F32', 'F33', 'F34', 'F39', 'F10', 'F11', 'F12', 
                           'F13', 'F14', 'F15', 'F16', 'F17', 'F18', 'F19', 'F40', 'F41', 'F42', 'F43', 
                           'F44', 'F45', 'F48', 'F50', 'F51', 'F52', 'F53', 'F54', 'F59', 'F60', 'F61', 
                           'F62', 'F63', 'F64', 'F65', 'F66', 'F68', 'F69', 'F70', 'F71', 'F72', 'F73', 
                           'F74', 'F75', 'F78', 'F79', 'F80', 'F81', 'F82', 'F83', 'F84', 'F88', 'F89', 
                           'F90', 'F91', 'F92', 'F93', 'F94', 'F95', 'F98', 'F99')

diagnoses['psych_flag'] = diagnoses['icd_code'].astype(str).str.startswith(psych_icd_codes)
psych_admissions = diagnoses[diagnoses['psych_flag']]['hadm_id'].unique()
admissions = admissions[admissions['hadm_id'].isin(psych_admissions)].copy()
diagnoses = diagnoses[diagnoses['hadm_id'].isin(admissions['hadm_id'])].copy()
print("Psych admissions:", admissions.shape[0], "Patients:", admissions['subject_id'].nunique(), "Notes:", diagnoses.shape[0])

# Sort admissions
admissions = admissions.sort_values(['subject_id', 'admittime']).copy()
print(admissions.head())
admissions.to_csv('./data/psych_admissions.csv', index=False)
diagnoses.to_csv('./data/psych_diagnoses.csv', index=False)

Initial admissions: 431088 Patients: 299777 Notes: 4752265
After excluding <18: 431088 Patients: 180747 Notes: 4752265
Psych admissions: 126867 Patients: 60782 Notes: 1543747
   subject_id   hadm_id            admittime            dischtime deathtime  \
0    10000032  22595853  2180-05-06 22:23:00  2180-05-07 17:15:00       NaN   
1    10000032  22841357  2180-06-26 18:27:00  2180-06-27 18:49:00       NaN   
3    10000032  29079034  2180-07-23 12:35:00  2180-07-25 17:55:00       NaN   
2    10000032  25742920  2180-08-05 23:44:00  2180-08-07 17:50:00       NaN   
4    10000068  25022803  2160-03-03 23:16:00  2160-03-04 06:26:00       NaN   

   admission_type      admission_location discharge_location insurance  \
0          URGENT  TRANSFER FROM HOSPITAL               HOME     Other   
1        EW EMER.          EMERGENCY ROOM               HOME  Medicaid   
3        EW EMER.          EMERGENCY ROOM               HOME  Medicaid   
2        EW EMER.          EMERGENCY ROOM            H

In [2]:
# Step 3: Define ICU readmissions
print("ICU admissions:", icustays.shape[0], "Patients:", icustays['subject_id'].nunique())
icu_stays = icustays['subject_id'].isin(admissions['subject_id']).copy()
icu_admissions = icustays.merge(admissions[['hadm_id', 'dischtime']], on='hadm_id', how='inner')
icu_admissions = icu_admissions.sort_values(['subject_id', 'intime']).copy()

icu_admissions['next_admission'] = icu_admissions.groupby('subject_id')['intime'].shift(-1)
icu_admissions['dischtime'] = pd.to_datetime(icu_admissions['dischtime'])
icu_admissions['next_admission'] = pd.to_datetime(icu_admissions['next_admission'])
icu_admissions.to_csv('./data/icu_psych_admissions.csv', index=False)

icu_admissions['readmitted_30d'] = (icu_admissions['next_admission'] - icu_admissions['dischtime']).dt.days <= 30
icu_admissions['readmitted_30d'] = icu_admissions['readmitted_30d'].fillna(False)
print("ICU Psych readmissions (30d):", icu_admissions['readmitted_30d'].sum(), 
    "Unique Patients:", icu_admissions[icu_admissions['readmitted_30d']]['subject_id'].nunique())

print(icu_admissions.head())
icu_admissions.to_csv('./data/icu_psy_readmissions.csv', index=False)


ICU admissions: 73141 Patients: 50934
ICU Psych readmissions (30d): 3363 Unique Patients: 2425
   subject_id   hadm_id   stay_id                       first_careunit  \
0    10000032  29079034  39553978   Medical Intensive Care Unit (MICU)   
1    10001217  24597018  37067082  Surgical Intensive Care Unit (SICU)   
2    10001217  27703517  34592300  Surgical Intensive Care Unit (SICU)   
3    10001884  26184834  37510196   Medical Intensive Care Unit (MICU)   
4    10002348  22725460  32610785                   Neuro Intermediate   

                         last_careunit               intime  \
0   Medical Intensive Care Unit (MICU)  2180-07-23 14:00:00   
1  Surgical Intensive Care Unit (SICU)  2157-11-20 19:18:02   
2  Surgical Intensive Care Unit (SICU)  2157-12-19 15:42:24   
3   Medical Intensive Care Unit (MICU)  2131-01-11 04:20:05   
4                   Neuro Intermediate  2112-11-30 23:24:00   

               outtime       los           dischtime      next_admission  \
0  21

In [3]:
# Step 4: Medication discontinuation
psych_meds = ['haloperidol', 'risperidone', 'quetiapine', 'olanzapine', 'lorazepam', 'diazepam', 'alprazolam',
              'fluoxetine', 'sertraline', 'citalopram', 'escitalopram', 'paroxetine', 'venlafaxine', 'duloxetine',
              'bupropion', 'mirtazapine', 'valproate', 'lithium']

# Total prescriptions before filtering
print("Total prescriptions:", prescriptions.shape[0], "Patients:", prescriptions['subject_id'].nunique())

# Filter prescriptions to adult psychiatric admissions
prescriptions = prescriptions[prescriptions['hadm_id'].isin(admissions['hadm_id'])].copy()
print("Prescriptions for psych admissions:", prescriptions.shape[0], "Patients:", prescriptions['subject_id'].nunique())
prescriptions.to_csv('./data/prescripted_patients.csv', index=False)

prescriptions['drug'] = prescriptions['drug'].astype(str).str.lower()
prescriptions['psych_med_flag'] = prescriptions['drug'].isin(psych_meds)
psych_prescriptions = prescriptions[prescriptions['psych_med_flag']].copy()
print("Psych prescriptions:", psych_prescriptions.shape[0], "Patients:", psych_prescriptions['subject_id'].nunique())
psych_prescriptions.to_csv('./data/psych_prescriptions.csv', index=False)

# Discontinued medications
psych_prescriptions.loc[:, 'stoptime'] = pd.to_datetime(psych_prescriptions['stoptime'], errors='coerce')
last_prescriptions = psych_prescriptions.groupby('hadm_id')['stoptime'].max().reset_index()
last_prescriptions.rename(columns={'stoptime': 'last_psych_med_time'}, inplace=True)

# Only include in admissions those hadm_id with psych meds
admissions_with_meds = admissions[admissions['hadm_id'].isin(last_prescriptions['hadm_id'])].copy()
admissions_with_meds = admissions_with_meds.merge(last_prescriptions, on='hadm_id', how='left')
admissions_with_meds['med_discontinued'] = admissions_with_meds['last_psych_med_time'].isnull()
discontinued_prescriptions = admissions_with_meds[admissions_with_meds['med_discontinued']]
print("Psych Medication discontinued admissions:", discontinued_prescriptions.shape[0])
print("Unique patients with psych medication discontinued:", discontinued_prescriptions['subject_id'].nunique())

Total prescriptions: 15399811 Patients: 158422
Prescriptions for psych admissions: 4706768 Patients: 50439
Psych prescriptions: 198927 Patients: 35130
Psych Medication discontinued admissions: 13
Unique patients with psych medication discontinued: 13


In [4]:
# To files
last_prescriptions.to_csv('./data/last_prescriptions_processed.csv', index=False)